In [7]:
import model
import pandas as pd
import numpy as np

In [90]:
def counterterror_model_http(request):
    """
    HTTP Cloud Function
    Args:
        request (flask.Request): request object providing parameters for model
    Returns:

    """

    #request_json = request.get_json(silent=True)
    #request_args = request.args
    request_json = request

    grid_size = {200:(2,3),300:(3,3),400:(3,4),500:(4,4),600:(4,5),\
                800:(5,5)}
    h, w = grid_size[request_json['starting_pop']]

    scenario = model.CounterterrorismModel(N=request_json['starting_pop'], \
            height=h,width=w,prob_violence=request_json['violence_prob'],\
            policy=request_json['gov_policy'].upper(),reactive_lvl=\
            request_json['reactive_level'].lower(),\
            discontent=request_json['discontent'].lower(),add_violence_aftermath=\
            request_json['add_violence_aftermath'],crowding_threshold=request_json['crowding_threshold'],\
            agent_birth_rate=request_json['agent_birth_rate'])
    for i in range(request_json['total_steps']):
        scenario.step()

    # create arrays to send to dashboard
    max_y_govtactions = []
    hMap_act_x = []
    hMap_act_y = []
    hMap_act_size = []
    hMap_act_symbol = []
    step = list(range(1,request_json['total_steps']+1))
    hMap_sympathy = []
    hMap_num_combats = []

    pChart_classes = []

    # ----- Create dominant sentiment and num_combatants grids --- #
    agents_df = scenario.datacollector.get_agent_vars_dataframe()
    agents_df = agents_df.reset_index()
    agents_nogov_df = agents_df[agents_df.status.isin(['anti-violence',\
        'combatant','neutral','sympathetic'])]
    agents_nogov_df['step_cat'] = agents_nogov_df.Step.astype('str')
    
    # create df of num agents of each status per step
    agent_stati_gb = agents_nogov_df.groupby(by=['step_cat','status'])
    agent_stati = agent_stati_gb['AgentID'].nunique().reset_index()
    agent_stati = agent_stati.rename(columns={'AgentID':'num_agents'})
    agent_stati['step'] = agent_stati.step_cat.astype('int')

    # create df of dominant sentiment per cell per step
    agent_sents_gb = agents_nogov_df.groupby(by=['step_cat','agent_loc','status'])
    sentiments_cnt = agent_sents_gb['AgentID'].nunique().reset_index()
    sentiments_cnt = sentiments_cnt.rename(columns={'AgentID':'num_agents'})
    max_sentiments_gb = sentiments_cnt.groupby(by=['step_cat','agent_loc'])
    max_sentiments = max_sentiments_gb['num_agents'].max().reset_index()
    #print('max sent shape',max_sentiments.shape)
    max_sentiments = max_sentiments.merge(sentiments_cnt, on=['step_cat','agent_loc','num_agents'])
    # randomly decide which to keep for now; ideally drop according to hierarchy in future iteration
    max_sents_nodups = max_sentiments.drop_duplicates(subset=['step_cat','agent_loc','num_agents'])
    max_sents_nodups['step'] = max_sents_nodups.step_cat.astype('int')
    sentiments_cnt['step'] = sentiments_cnt.step_cat.astype('int')

   # ----- Create govt action arrays ----- #
    govt_actions = scenario.datacollector.get_table_dataframe('govt_actions')
    max_num_govt_actions = 0   
    full_ys = ['A','B','C','D','E','F']
    scen_ys = full_ys[:scenario.grid.width]
    full_xs = ["ㅤ1", "ㅤ2", "ㅤ3","ㅤ4", "ㅤ5", "ㅤ6"]        
    scen_xs = full_xs[:scenario.grid.height]     

    for mod_step in range(1,request_json['total_steps']+1):
        print('mod step',mod_step)
        agent_step_df = agent_stati[agent_stati.step == mod_step]

        # update pChart classes for each step
        pChart_classes_step = [0,0,0,0]
        stati_map = {'anti-violence':0.0,'neutral':0.25,
                'sympathetic':0.50,'combatant':0.75}
        stati = ['anti-violence','neutral','sympathetic','combatant']
        for idx, stati_cnt in enumerate(stati):
            # if agents of status exist, set to number, else assign 0
            if stati_cnt in list(agent_step_df.status):
                pChart_classes_step[idx] = agent_step_df[agent_step_df.status==stati_cnt].\
                    num_agents.item()
        
        #print('pChart classes to add',pChart_classes_step)
        pChart_classes.append(pChart_classes_step)

        # update dominant sentiments and num combatants per step
        max_sent_step_df = max_sents_nodups[max_sents_nodups.step == mod_step]
        hmap_sentiments = np.zeros((scenario.grid.width, scenario.grid.height))
        hmap_num_combatants = np.zeros((scenario.grid.width, scenario.grid.height))
        for hmap_loc in max_sent_step_df.agent_loc:
            hmap_sentiments[hmap_loc] = stati_map[max_sent_step_df[max_sent_step_df.agent_loc==\
                                        hmap_loc].status.item()]

            
            test_comb_df = sentiments_cnt[(sentiments_cnt.step == mod_step) & \
                        (sentiments_cnt.agent_loc == hmap_loc)]
            if 'combatant' in list(test_comb_df.status):
                hmap_num_combatants[hmap_loc] = test_comb_df[test_comb_df.status == \
                                    'combatant'].num_agents.item()
            else:
                hmap_num_combatants[hmap_loc] = 0
        
        #print('hMap sympathy to append',hmap_sentiments)
        hMap_sympathy.append(hmap_sentiments.tolist())

        #print('hMap num combatants to append',hmap_num_combatants)
        hMap_num_combats.append(hmap_num_combatants.tolist())


        # update gov't action per step
        hmap_x = []
        hmap_y = []
        hmap_size = []
        hmap_symbol = []
        govt_action_df = govt_actions[govt_actions.step == mod_step]
        this_step_action = govt_action_df.govt_action.item()
        #print('this step action',this_step_action)
        if this_step_action in ['INDISC-CONC','INDISC-REPR']:
            max_num_govt_actions += 1
            hmap_y_long = [[y] * scenario.grid.height for y in scen_ys]
            hmap_y = [val for sublist in hmap_y_long for val in sublist]
            hmap_x = scen_xs * scenario.grid.width
            hmap_size = [80] * scenario.grid.width * scenario.grid.height
            if this_step_action == 'INDISC-CONC':
                hmap_symbol = [105] * scenario.grid.width * scenario.grid.height
            elif this_step_action == 'INDISC-REPR':
                hmap_symbol = [106] * scenario.grid.width * scenario.grid.height

        elif this_step_action in ['TARG-CONC','TARG-REPR']:
            max_num_govt_actions += 1
            y, x = govt_action_df.action_loc.item()
            hmap_x = [scen_xs[x]]
            hmap_y = [scen_ys[y]]
            hmap_size = [50]
            if this_step_action == 'TARG-CONC':
                hmap_symbol = [105]
            elif this_step_action == 'TARG-REPR':
                hmap_symbol = [106]

        max_y_govtactions.append(max_num_govt_actions)
        hMap_act_x.append(hmap_x)
        hMap_act_y.append(hmap_y)
        hMap_act_size.append(hmap_size)
        hMap_act_symbol.append(hmap_symbol)


    # ---- Create lineplot agents and attacks arrays --- #
    
    model_df = scenario.datacollector.get_model_vars_dataframe()
    
    cummax_model_df = model_df.cummax()
    max_y_agentattacks = list(cummax_model_df.max(axis=1))
    lPlot_nAttacks = list(model_df['num_attacks'])
    lPlot_nAgents = list(model_df['num_agents'])


    # ---- Add gov't cumulative actions ----- #

    govt_status_df = agents_df[agents_df.status.isin(['NONE','INDISC-REPR','INDISC-CONC','TARG-CONC','TARG-REPR'])].reset_index()
    govt_status_df.drop('AgentID', axis=1,inplace=True)
    govt_status_df = govt_status_df.sort_values('Step',ascending=True)
    govt_status_cum = govt_status_df.groupby(['status']).cumcount()
    govt_status_cum = govt_status_cum.rename('cumulative_actions')
    govt_status_cum = govt_status_df.join(govt_status_cum)
    # cumcount starts at 0, so rise to 1
    govt_status_cum['cumulative_actions'] = govt_status_cum['cumulative_actions'] + 1
    govt_status_cum = pd.get_dummies(govt_status_cum['status']).cumsum()

    lPlot_cumTargRepr = list(govt_status_cum['TARG-REPR'])
    lPlot_cumIndiscRepr = list(govt_status_cum['INDISC-REPR'])
    lPlot_cumTargConc = list(govt_status_cum['TARG-CONC'])
    lPlot_cumIndiscConc = list(govt_status_cum['INDISC-CONC'])

    return_vals = {'step':step,
                    'lPlot_nAttacks':lPlot_nAttacks,
                    'lPlot_nAgents':lPlot_nAgents,
                    'hMap_act_x':hMap_act_x,
                    'hMap_act_y':hMap_act_y,
                    'hMap_act_size':hMap_act_size,
                    'hMap_act_symbol':hMap_act_symbol,
                    'max_y_agentattacks':max_y_agentattacks,
                    'max_y_govtactions':max_y_govtactions,
                    'hMap_sympathy':hMap_sympathy,
                    'hMap_num_combats':hMap_num_combats,
                    'pChart_classes':pChart_classes,
                    'lPlot_cumTargRepr':lPlot_cumTargRepr,
                    'lPlot_cumIndiscRepr':lPlot_cumIndiscRepr,
                    'lPlot_cumTargConc':lPlot_cumTargConc,
                    'lPlot_cumIndiscConc':lPlot_cumIndiscConc}

    return return_vals

In [86]:
sim_json = {
        "violence_prob": 0.0005,
        "gov_policy": "None",
        "reactive_level": "Low",
        "discontent": "Mid",
        "starting_pop": 200,
        "total_steps": 400,
        "add_violence_aftermath":10,
        "crowding_threshold":30, 
        "agent_birth_rate":0.03
    }

In [91]:
test_resp = counterterror_model_http(sim_json)

/tmp/ipykernel_32182/2331464870.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agents_nogov_df['step_cat'] = agents_nogov_df.Step.astype('str')
/tmp/ipykernel_32182/2331464870.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_sents_nodups['step'] = max_sents_nodups.step_cat.astype('int')


mod step 1
mod step 2
mod step 3
mod step 4
mod step 5
mod step 6
mod step 7
mod step 8
mod step 9
mod step 10
mod step 11
mod step 12
mod step 13
mod step 14
mod step 15
mod step 16
mod step 17
mod step 18
mod step 19
mod step 20
mod step 21
mod step 22
mod step 23
mod step 24
mod step 25
mod step 26
mod step 27
mod step 28
mod step 29
mod step 30
mod step 31
mod step 32
mod step 33
mod step 34
mod step 35
mod step 36
mod step 37
mod step 38
mod step 39
mod step 40
mod step 41
mod step 42
mod step 43
mod step 44
mod step 45
mod step 46
mod step 47
mod step 48
mod step 49
mod step 50
mod step 51
mod step 52
mod step 53
mod step 54
mod step 55
mod step 56
mod step 57
mod step 58
mod step 59
mod step 60
mod step 61
mod step 62
mod step 63
mod step 64
mod step 65
mod step 66
mod step 67
mod step 68
mod step 69
mod step 70
mod step 71
mod step 72
mod step 73
mod step 74
mod step 75
mod step 76
mod step 77
mod step 78
mod step 79
mod step 80
mod step 81
mod step 82
mod step 83
mod step 84
m

In [92]:
test_resp

{'step': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
 

In [93]:
import json
with open("long_sim_response.json","w") as f:
    json.dump(test_resp, f)

In [83]:
scenario = model.CounterterrorismModel(N=200, height=3,width=4,prob_violence=0.003,policy='NONE',reactive_lvl='none',discontent='mid')
for i in range(700):
    scenario.step()

In [82]:
from random import choices
prob_violence = 0.0001
choices([True, False], weights=\
            [prob_violence, (1 - prob_violence)], k=1)

[False]

In [84]:
model_df = scenario.datacollector.get_model_vars_dataframe()
model_df.head()

,num_agents,num_attacks
0,200,58
1,144,59
2,147,59
3,148,59
4,151,59
